# All useful functions called in `results`

In [1]:
import re
import pycountry
import pycountry_convert as pc
#from country_named_entity_recognition import find_countries
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import numpy as np

In [2]:
def extract_string(string):
    """
    Function to extract all words between quotes

    Parameters: string (str) - a string containing words between quotes

    Returns: between_quotes (list) - a list of words between quotes
    """
    # The re library allows to extract all words between quotes
    between_quotes = re.findall(r'"(.*?)"', string)
    # Suppressing all words starting with /, as they are not country names
    for word in between_quotes:
        if word[0] == '/':
            between_quotes.remove(word)
    # Returning the list of words between quotes
    return between_quotes

In [3]:
def estimate_release_year(row, mean_release_year_by_genre):
    """
    Function to estimate the release year of a movie based on its genres (by taking the average of the release years of movies with the same genres)

    Parameters: row (pd.Series) - a row of a pandas DataFrame

    Returns: release_year (int) - the estimated release year of the movie
    """
    if pd.isna(row['Movie_release_date']):
        genres = row['genre_list']
        mean_years = [mean_release_year_by_genre[genre] for genre in genres if genre in mean_release_year_by_genre]
        if mean_years:
            return sum(mean_years) / len(mean_years)
        else:
            return pd.NA
    else:
        return row['Movie_release_date']

In [4]:
def count_countries(string):
    """
    Function to count the number of countries in a string

    Parameters: string (str) - a string containing countries separated by commas

    Returns: country_number (int) - the number of countries
    """
    # Counting of comas in the string
    country_number=string.count(',') +1 
    # Returning the number of countries
    return country_number

In [5]:
def is_dict_string(s):
    """
    Vérifie si une chaîne de caractères peut être convertie en dictionnaire.
    
    Paramètres:
    s (str): La chaîne de caractères à vérifier.
    
    Retourne:
    bool: True si la chaîne peut être convertie en dictionnaire, False sinon.
    """
    try:
        ast.literal_eval(s)
        return True
    except (ValueError, SyntaxError):
        return False

In [6]:
def get_word_count(url, retries=3, delay=5):
    for attempt in range(retries):
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content of the page
        content = soup.find('div', {'id': 'mw-content-text'})

        if content:
            # Extract text and remove citations
            text = re.sub(r'\[\d+\]', '', content.get_text())
            
            # Count the number of words
            words = text.split()
            return len(words)
        else:
            return None


In [7]:
# Function to preprocess event labels
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)  # Remove non-alphanumeric characters
    return text

In [1]:
def text_split(text) :

    #Remove from text all punctuations
    text = re.sub(r'[^\w\s]', '', text)

    #Tokenise the text in words and convert in small letters
    words = word_tokenize(text)
    
    #Filter the words depending on their grammatical class
    
    text_splited = []

    for word in words:

        #Find the grammatical class (pos) of a word and tag it with it
        pos = pos_tag([word])[0][1]
        #Keep only the nouns (singular and plural)
        if pos in ['NN', 'NNS']: #and word not in stop_words:
            text_splited.append(word)
    
    return text_splited

In [2]:
def bag_of_words(text_splited) : 
    
    #Create a dictionary mapping each word to a unique id
    dictionary = Dictionary([text_splited])
    
    
    corpus = dictionary.doc2bow(text_splited)
    
    return corpus, dictionary

In [3]:
def lda(df_line) : 
    corpus, dictionary = df_line['corpus']
    
    lda_model = LdaModel([corpus], num_topics=1, id2word=dictionary)
    return lda_model

In [4]:
def get_topics(lda_model):

    #Provide 50 words that are the most common in the given set
    return lda_model.show_topics(num_topics=1, num_words=50, formatted=True)
    
    topic_words = []

    for topic in topics:
        for word, freq in topic[1]:
            topic_words.append({'topic': topic[0], 'word': word, 'frequency': freq})

    return topic_words

In [5]:
def extract(data):

    serie = data[0][0][1] 
    
    #Get the words and their weight from the serie
    matches = re.findall(r'([\d.]+)\*"(.*?)"', serie) 
    freq_df = pd.DataFrame(matches, columns=["freq", "word"])

    #Convert the frequency in floats
    freq_df["freq"] = freq_df["freq"].astype(float) 
    
    return freq_df

In [13]:
def text_split_new(text) :

    #Initialize a set of frequent and not relevent words in english
    #stop_words = set(stopwords.words('english'))

    #Remove from text all punctuations
    text = re.sub(r'[^\w\s]', '', text)
    

    #Tokenise the text in words and convert in small letters
    words = word_tokenize(text)
    
    #Filter the words depending on their grammatical class
    
    text_splited = []

    for word in words:

        #Find the grammatical class (pos) of a word and tag it with it
        pos = pos_tag([word])[0][1]
        #Keep only the nouns (singular and plural)
        if pos in ['NN', 'NNS']: #and word not in stop_words:
            text_splited.append(word)
    
    return text_splited

In [14]:
def lda_new(df_line):
    corpus, dictionary = df_line  # Unpacking the tuple directly
    lda_model = LdaModel([corpus], num_topics=1, id2word=dictionary)
    return lda_model

In [15]:

def get_topics_new(lda_model):
    #provid fifty words belonging to a single topic
    return lda_model.show_topics(num_topics=1, num_words=50, formatted=True)


In [16]:
def extract_new(topics_tuple):
    topics = []
    
    #Get the words and their weight from the serie
    pattern = re.compile(r'(\d+\.\d+)\*"(.*?)"')
    for topic_id, topics_str in topics_tuple:
        matches = pattern.findall(topics_str)
        for match in matches:
            freq = float(match[0]) #Convert the frequency in floats
            word = match[1]
            topics.append((word, freq))
    return topics

In [17]:
# Function to read and preprocess text data
def read_and_preprocess(file_path):
    """
    Reads a text file, removes newline characters, and tokenizes it into sentences.
    
    Args:
        file_path (str): Path to the input text file.
    
    Returns:
        list: A list of stripped sentences.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        text=text.replace('\n','')

    # Tokenize the text into sentences using NLTK
    sentences = nltk.sent_tokenize(text)  
    return [sentence.strip() for sentence in sentences if sentence.strip()] 

In [6]:
def LDA_applicator(summaries):

    #Group the summaries of one decade together
    all_summaries = summaries[0].str.cat(sep=' ')
    summary_df = pd.DataFrame({"summary": [all_summaries]})

    #Process and apply the LDA on the summaries
    summary_df['split'] = summary_df['summary'].apply(text_split)
    summary_df['corpus'] = summary_df['split'].apply(bag_of_words)
    summary_df['lda'] = summary_df.apply(lda, axis=1)
    
    #Extract the words and their frequenciesin a DataFrane
    word_freq = summary_df['lda'].apply(get_topics)
    topics = extract(word_freq)
    topics.set_index('word', inplace=True)
    return topics

In [7]:
def LDA_plots(topic, decade):
    
    #Plotting in the form of barplots
    plt.figure(figsize=(10, 6))
    topic.plot(kind='bar', legend=False)

    plt.title('Most common words in summaries of films in the ' + decade +'s')
    plt.xlabel('Words')
    plt.ylabel('Frequency')

    #To extract the barplot, uncomment this line:
    #plt.savefig(f'LDA_barplot_{decade}.png', dpi=300, bbox_inches='tight')
    plt.show()

    #Plotting in the form of word clouds
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate_from_frequencies(topic['freq'])

    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  
    plt.title('Word Cloud of most common words in the ' + decade+ 's')
    
    #To extract the wordcloud, uncomment this line:
    #plt.savefig(f'LDA_wordcloud_{decade}.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def read_and_preprocess_per_movie(file_path, column_index=0, delimiter='\n'):
    with open(file_path, 'r', encoding='utf-8') as file:
        movies_summary = [line.strip() for line in file.readlines() if line.strip()] 
    return movies_summary

In [8]:
def export_clusters_and_pca(embeddings, movies, decade, n_clusters=3):
    #Clustering the movies
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings.cpu().numpy())

    #Extracting the index of the movies in the cluster
    clustered_movies = {i: [] for i in range(n_clusters)}
    for idx, cluster_id in enumerate(cluster_labels):
        clustered_movies[cluster_id].append(movies[idx])

    movie_per_cluster = ''
    for cluster_id, cluster in clustered_movies.items():
        for mov in cluster:
            movie_per_cluster += mov
        movie_per_cluster += '\n'
    
    with open(f'extraction_test/clusters_per_movie_{decade}.txt', "w") as file:
        file.write(movie_per_cluster)

    #Extracting the PCA
    pca = PCA(n_components=2)
    pca_embeddings = pca.fit_transform(embeddings.cpu().numpy())
    pca_df = pd.DataFrame(pca_embeddings, columns=['PCA Component 1', 'PCA Component 2'])
    pca_df.to_csv(f'extraction_test/pca_{decade}.csv', index=False)

In [9]:
def plot_clusters_and_pca(embeddings, decade, cluster_names, n_clusters=3):

    #Clustering the movies
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings.cpu().numpy())

    #Apply the PCA
    pca = PCA(n_components=2)
    pca_embeddings = pca.fit_transform(embeddings.cpu().numpy())

    #Plotting the PCA with the clustering, using the names found with LLAMa
    plt.figure(figsize=(10, 6))
    for cluster in range(n_clusters):
        cluster_points = pca_embeddings[cluster_labels == cluster]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'{cluster_names[cluster]}' )

    plt.title(f'Clusters of {decade}s Movies')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.legend()
    plt.show()